## АвтоГДИ. Метод MDH
Непосредственной задачей, стоящей перед гидродинамическими
исследованиями пластов и скважин, является получение данных, характеризующих
их гидродинамические свойства.
При проведении гидродинамических исследований нефтегазовых пластов необходимо решать две задачи.
1. Прямая задача подземной гидродинамики состоит в определении давления в
произвольной точке пласта с известными коллекторскими и геометрическими
параметрами при работе возмущающей скважины с дебитом, изменяющемся по
заданному закону или с заданным на скважине переменным давлением.
2. Обратная задача подземной гидродинамики состоит в определении
коллекторских и геометрических параметров пласта по кривым изменения дебита и
давления, замеренным при исследованиях.
<br>
<br>

Решение этих двух задач осуществляется с помощью формул подземной
гидродинамики, связывающих давление, дебит и параметры пласта. Получаемые
решения должны иметь вид зависимости изменения давления $\Delta P$ в произвольной
точке пласта от времени $t$, расстояния $r$ от возмущающей скважины до исследуемой
точки, дебита скважины $Q(t)$, фильтрационных параметров пласта –
гидропроводност $kh/μ$ и пьезопроводности $χ$ , а также геометрических параметров пласта.

In [15]:
import math
from typing import List, Any
from math import log10
from excel_reader import hydrodynamic_data
from Hydrodynamic_Data import HydrodynamicData
import numpy as np
from excel_reader import formation_info
from Fluid_info import FormationInfo
import typing as tp

На вход подаются измерения со скважины (данные КВД), а именно изменения давления и изменение времени после остановки скважины. Необходимо преобразовать данные по времени в формат $Lg(t)$.
<br>
Функция вычисления десятичного логарифма от значений массива $\Delta t$.

In [16]:
def calculate_lg_t(hydrodynamic_data: HydrodynamicData) -> List[float]:
    delta_t_values = hydrodynamic_data.delta_t
    lg_t_values = [log10(value * 3600) for value in delta_t_values]
    return lg_t_values

При обработке кривой восстановления давления методом МДХ (или методом касательной) без учета притока жидкости в скважину, упрощенное решение основного уравнения имеет вид:
<div style="text-align:center">
$\large\Delta P(t) = P_c(t) - P_c(0) = \frac{Q_0 \mu}{4\pi k h} \ln\left(2.25 \frac{χ t}{r_{c. пр.}^2}\right) = \frac{2.3 Q_0 \mu}{4\pi k h} \lg\left(2.25 \frac{χ t}{r_{c. пр.}^2}\right)$
</div>

Предыдущее уравнение запишем в виде:
<div style="text-align:center">
$\large\Delta P(t) = \frac{2.3 Q_0 \mu}{4\pi k h} \lg\left(\frac{2.25χ}{r_{c. пр.}^2}\right) + \frac{2.3 Q_0 \mu}{4\pi k h} \lg t$
</div>
<br>
Введем обозначения:
<div style="text-align:center">
$\large\ i = \frac{2.3 Q_0 \mu}{4\pi k h}, \quad A = \frac{2.3 Q_0 \mu}{4\pi k h} \lg\left(\frac{2.25χ}{r_{c. пр.}^2}\right)$.
</div>
Тогда выражение для $\Delta P(t)$ представляется в виде:
<div style="text-align:center">
$\large\Delta P(t) = A + i \lg t$
</div>
<br>
<br>
На основании данных об изменении давления и времени строится кривая восстановления давления (КВД) в полулогарифмических координатах, где ось абсцисс представляет собой логарифм времени $lg (t)$, а ось ординат - изменение давления $\Delta P(t)$.
</div>
На прямолинейном участке кривой восстановления давления произвольно выбираются две точки с координатами $\Delta P(t_1)$, $lg (t_1)$ и $\Delta P(t_2)$, $log (t_2)$, которые определяют значение угла наклона этого участка.
<br>
<br>
Угол наклона прямолинейного участка кривой восстановления давления вычисляется как отношение разности изменений давления между выбранными точками к разности логарифма от времени между выбранными точками:
<div style="text-align:center">
$\large\ i = \text{tg}\alpha = \frac{\Delta P(t_2) - \Delta P(t_1)}{\lg t_2 - \lg t_1}$
</div>
<br>
Коэффициент $A$ находится как:
<div style="text-align:center">
$A = \Delta P_1 - i \cdot \lg t_1$

Определяется гидропроводность пласта в районе исследуемой скважины:
<div style="text-align:center">
$\large\varepsilon = \frac{k h}{\mu} = \frac{2.3 Q_0 b}{4\pi}$

Определяется комплексный параметр:
<div style="text-align:center">
$\large\frac{\chi}{r_{c.пр.}^2} = \frac{{10^{\frac{A}{i}}}}{2.25} $

В том случае, если интерпретатор имеет удовлетворяющие его значения эффективной толщины пласта $h$, пористости $m$, коэффициента сжимаемости $ C_{\text{total}}$, то можно произвести оценку ряда параметров.

Оценим коэффициент пьезопроводности:
<div style="text-align:center">
$\large\chi = \frac{\varepsilon}{{h \cdot C_{\text{total}}}}$

Вычисляется коэффициент продуктивности при радиусе контура питания, равном половине расстояния между скважинами:
<div style="text-align:center">
$\large\ K = \frac{{Q_0}}{{P_{\text{пл.}} - P_c}} = \frac{{\frac{{2\pi kh}}{{\mu}}}}{{2.3 \lg\left(\frac{{R_k}}{{r_{c.пр.}}}\right)}}$


Функция вычисления основных параметров по методу MDH.

In [17]:
def calculate_MDH(formation_info: FormationInfo) -> tp.Tuple[float, float, float, float, tp.Union[float, int]]:
    Q_0 = formation_info.Q
    b = formation_info.b
    density = formation_info.density
    h = formation_info.h
    mu = formation_info.mu
    c_total = formation_info.c_total
    distance = formation_info.distance
    epsilon = (2.3 * Q_0 * 11.57 * b) / (4 * np.pi * i * density)
    param = (10 ** (A / i)) / 2.25
    piezo = epsilon / ((h * 100) * c_total)
    r_pr = math.sqrt(piezo / param)
    K = (2 * np.pi * epsilon) / (2.3 * log10(((distance * 100) / 2) / r_pr))
    K_prod = (K * density) / (11.57 * b)
    k = (epsilon * mu) / (h * 100) * 10 ** 3
    return round(epsilon, 3), round(param, 3), round(piezo, 3), round(K_prod, 3), round(k, 3)


Вызов функций и вывод полученных результатов

In [18]:
lg_t_result = calculate_lg_t(hydrodynamic_data)
delta_p = hydrodynamic_data.delta_p
delta_p2 = delta_p[-1]
delta_p1 = delta_p[-9]
lg_t2 = lg_t_result[-1]
lg_t1 = lg_t_result[-9]
i = (delta_p2 - delta_p1) / (lg_t2 - lg_t1)
A = delta_p1 - i * lg_t1
epsilon, param, piezo, K_prod, k = calculate_MDH(formation_info)
print(f"Гидропроводность пласта: {epsilon}, (Д*см)/cП")
print(f"Комплексный параметр: {param}, 1/c")
print(f"Пьезопроводность пласта: {piezo} cм^2/c")
print(f"Коэффициент продуктивности: {K_prod}, т/(сут*атм)")
print(f"Проницаемость: {k}, мД")

Гидропроводность пласта: 3.439, (Д*см)/cП
Комплексный параметр: 0.231, 1/c
Пьезопроводность пласта: 29.543 cм^2/c
Коэффициент продуктивности: 0.24, т/(сут*атм)
Проницаемость: 8.579, мД
